### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [1]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    token = "", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


C:\Users\trini\Python\MTAA\Proyecto_Final\unsloth_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


C:\Users\trini\Python\MTAA\Proyecto_Final\unsloth_env\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:339: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.7.3: Fast Gemma3 patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 3070 Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
import torch
print("CUDA disponible:", torch.cuda.is_available())
print("Versión CUDA compilada:", torch.version.cuda)
print("Dispositivo:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No detectado")


CUDA disponible: True
Versión CUDA compilada: 12.8
Dispositivo: NVIDIA GeForce RTX 3070 Laptop GPU


We now add LoRA adapters so we only need to update a small amount of parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [4]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train[:10]")

In [5]:
dataset[1]

{'conversations': [{'from': 'human',
   'value': 'Explain how recursion works and provide a recursive function in Python that calculates the factorial of a given number.'},
  {'from': 'gpt',
   'value': "Recursion is a programming technique where a function calls itself to solve a problem. It breaks down a complex problem into smaller, more manageable subproblems until a base case is reached. The base case is a condition where the function does not call itself, but instead returns a specific value or performs a specific action.\n\nIn the case of calculating the factorial of a number, recursion can be used to break down the problem into simpler subproblems. The factorial of a non-negative integer n is the product of all positive integers less than or equal to n.\n\nHere is a recursive function in Python that calculates the factorial of a given number:\n\n```python\ndef factorial(n):\n    # Base case: factorial of 0 or 1 is 1\n    if n == 0 or n == 1:\n        return 1\n    # Recursive c

In [6]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [7]:
import pandas as pd
df = pd.read_csv('Drug-ATC.csv', encoding="utf-8")
SMILES = df['Neutralized SMILES']
ATC4 = df['ATC Codes']

In [8]:
for i in range(len(SMILES)):
    smiles = SMILES.at[i]
    s_smiles = smiles.replace("\\", "\\\\")
    SMILES.at[i] = s_smiles

In [9]:
SMILES

0       CC[C@H](C)[C@H](NC(=O)[C@H](CCC(=O)O)NC(=O)[C@...
1       CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2       CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
3       N=C(N)NCCC[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H]...
4       CC(=O)N[C@H](Cc1ccc2ccccc2c1)C(=O)N[C@H](Cc1cc...
                              ...                        
3584                   CC=C1CC2CNc3cc(O)c(OC)cc3C(=O)N2C1
3585               CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC(=O)O
3586                        CCCCCCCCC1=C(CCCCCCCC(=O)O)C1
3587    COCC1OC(OC2C(COC)OC(OC3C(COC)OC(OC)C(OC)C3OC)C...
3588    O=C(O)c1cc(O)c2c(c1)C(C1c3cc(C(=O)O)cc(O)c3C(=...
Name: Neutralized SMILES, Length: 3589, dtype: object

In [10]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    SMILES, ATC4, test_size=0.2, random_state=42)

In [11]:
from datasets import Dataset
data  = []
for SMILES, ATCcode in zip(X_train, y_train):
    data.append({
        'conversations': [
            {'from': 'human', 'value': SMILES},
            {'from': 'gpt', 'value': ATCcode}
        ],
        'source': 'dataset',  
        'score': 1.0 
    })

dataset = Dataset.from_list(data)
print(dataset)

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 2871
})


In [12]:
dataset[4]

{'conversations': [{'from': 'human',
   'value': 'C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@@H](O)[C@@H]2O'},
  {'from': 'gpt', 'value': 'G03CA; G03CC'}],
 'source': 'dataset',
 'score': 1.0}

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [13]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=16): 100%|█████████████████████████| 2871/2871 [00:47<00:00, 60.28 examples/s]


Let's see how row 100 looks like!

In [14]:
dataset[4]

{'conversations': [{'content': 'C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@@H](O)[C@@H]2O',
   'role': 'user'},
  {'content': 'G03CA; G03CC', 'role': 'assistant'}],
 'source': 'dataset',
 'score': 1.0}

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [15]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map: 100%|████████████████████████████████████████████████████████████████| 2871/2871 [00:00<00:00, 7844.88 examples/s]


Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [16]:
dataset[100]["text"]

'<start_of_turn>user\nCC(C)c1ccccc1OCC1=NCCN1<end_of_turn>\n<start_of_turn>model\nR01AA<end_of_turn>\n'

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [17]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "False"

In [18]:
original_map = Dataset.map

def patched_map(self, *args, **kwargs):
    kwargs["num_proc"] = 1
    return original_map(self, *args, **kwargs)

Dataset.map = patched_map

In [19]:
for i, row in enumerate(dataset):
    try:
        tokenizer(row["text"])
    except Exception as e:
        print(f"Error en el ejemplo {i}: {e}")
        break

In [20]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]: 100%|██████████████████████████████████████| 2871/2871 [00:00<00:00, 14232.23 examples/s]


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [21]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map: 100%|███████████████████████████████████████████████████████████████| 2871/2871 [00:00<00:00, 17088.44 examples/s]


Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [22]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nCC(C)c1ccccc1OCC1=NCCN1<end_of_turn>\n<start_of_turn>model\nR01AA<end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [23]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                       R01AA<end_of_turn>\n'

In [24]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3070 Laptop GPU. Max memory = 8.0 GB.
4.385 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [25]:
import os
os.environ["TORCH_COMPILE_DISABLE"] = "1"
import torch._dynamo
torch._dynamo.config.disable = True

In [26]:
print(torch.__version__)
print(torch.version.cuda)
print(torch._dynamo.config.cache_size_limit)
print(torch._dynamo.config.repro_after)

2.7.1+cu128
12.8
1024
None


In [27]:
print(model.__class__)

<class 'peft.peft_model.PeftModelForCausalLM'>


In [28]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,871 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248 of 4,314,980,720 (0.35% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,9.904300
2,12.579300
3,13.716600
4,11.492600
5,9.529700
6,6.487000
7,5.734800
8,4.989300
9,4.637000
10,3.701500


In [29]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

93.7275 seconds used for training.
1.56 minutes used for training.
Peak reserved memory = 5.73 GB.
Peak reserved memory for training = 1.345 GB.
Peak reserved memory % of max memory = 71.625 %.
Peak reserved memory for training % of max memory = 16.812 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [30]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : X_test[2830], #N06BX
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nCc1ncc(COC(=O)CCC(=O)OCCN(C)C)c(CO)c1O<end_of_turn>\n<start_of_turn>model\nP01AA<end_of_turn>']

In [31]:
print(X_test[2830])
print(y_test[2830])

Cc1ncc(COC(=O)CCC(=O)OCCN(C)C)c(CO)c1O
N06BX


In [32]:
X_test

2830               Cc1ncc(COC(=O)CCC(=O)OCCN(C)C)c(CO)c1O
315     NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3...
2067                                C/C(O)=N\\c1ccc(O)cc1
3536    CC1(C(=O)O)CCC2(C)CCC3(C)C(=CC(=O)C4C5(C)CCC(O...
439               COC(=O)Nc1nc2ccc(C(=O)c3ccccc3)cc2[nH]1
                              ...                        
2064    C[C@@](O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C#...
288             CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)nc21
2812    CC(=O)N[C@@H](CC(=O)N[C@@H](CCC(=O)O)C(=O)O)C(...
3155            C[C@@H]1C[C@@H](OC(=O)c2cccnc2)CC(C)(C)C1
2230                  CN(C)CC[C@@H](c1ccc(Cl)cc1)c1ccccn1
Name: Neutralized SMILES, Length: 718, dtype: object

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [33]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : X_test[2230],}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

L03AB<end_of_turn>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [34]:
X_train.reset_index(drop=True)
X_train.to_csv("X_train_finetuning.csv",index=False)

In [35]:
X_test.reset_index(drop=True)
X_test.to_csv("X_test_finetuning.csv",index=False)

In [36]:
y_train.reset_index(drop=True)
y_train.to_csv("y_train_finetuning.csv",index=False)
y_test.reset_index(drop=True)
y_test.to_csv("y_test_finetuning.csv",index=False)

In [37]:
model.save_pretrained("gemma-3-finetune")  # Local saving
tokenizer.save_pretrained("gemma-3-finetune")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

['gemma-3-finetune\\processor_config.json']

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [38]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)nc21",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

C003; C002<end_of_turn>


### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [40]:
if True: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

Found HuggingFace hub cache directory: C:\Users\trini\.cache\huggingface\hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|█████████████████████████████████████████████| 2/2 [40:03<00:00, 1201.88s/it]


If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [42]:
import os
print(os.environ["PATH"])

C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files\Git\cmd;C:\ProgramData\ComposerSetup\bin;C:\Program Files\nodejs\;C:\Program Files\Docker\Docker\resources\bin;C:\Program Files\NVIDIA Corporation\Nsight Compute 2022.3.0\;C:\Program Files\NVIDIA Corporation\NVIDIA app\NvDLISR;C:\Users\trini\AppData\Local\Microsoft\WindowsApps;C:\Users\trini\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\trini\AppData\Local\GitHubDesktop\bin;C:\Users\trini\AppData\Roaming\Composer\vendor\bin;C:\Users\trini\php;C:\Users\trini\AppData\Roaming\npm;C:\

In [43]:
if True: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "q8_0", # For now only Q8_0, BF16, F16 supported
    )

'python' is not recognized as an internal or external command,
operable program or batch file.
'python3' is not recognized as an internal or external command,
operable program or batch file.


RuntimeError: Unsloth: Failed to convert llama.cpp/unsloth_convert_hf_to_gguf.py to GGUF.

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

In [ ]:
# Save to 8bit Q8_0
# model.save_pretrained_gguf("finetuned_model8bit", tokenizer,)
# Save to 16bit GGUF
if True: model.save_pretrained_gguf("gemma-3", tokenizer, quantization_method= "f16")

# Save to q4_k_m GGUF
model.save_pretrained_gguf("model", tokenizer, quantization_type = "q4_k_m")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
